# EJERCICIO GUIADO 1   

## DATASCIENCESTER - La red de los Científicos de Datos   

### Construyendo nuestra Red   

Es el primer día de trabajo en DataSciencester, y el vicepresidente de Redes tiene muchas preguntas sobre los usuarios. Hasta ahora no tenía nadie a quien preguntar, así que está muy emocionado de tener alguien nuevo en el
equipo.   
En particular, le interesa identificar quiénes son los “conectores clave” de todos los científicos de datos. Para ello proporciona un volcado de la red completa de DataSciencester (en la vida real, la gente no suele pasar los datos que uno necesita).   
¿Qué aspecto tiene este volcado de datos? Consiste en una lista de usuarios, cada uno representado por un dict que contiene su id (que es un número) y su name (que,en una de esas fabulosas conjunciones planetarias, concuerda con su id):

In [ ]:
users = [
    { "id": 0, "name": "Hero" },
    { "id": 1, "name": "Dunn" },
    { "id": 2, "name": "Sue" },
    { "id": 3, "name": "Chi" },
    { "id": 4, "name": "Thor" },
    { "id": 5, "name": "Clive" },
    { "id": 6, "name": "Hicks" },
    { "id": 7, "name": "Devin" },
    { "id": 8, "name": "Kate" },
    { "id": 9, "name": "Klein" }
]

También ofrece los datos de “amistad” (friendship), representados como una lista de pares de identificadores:

In [ ]:
friendship_pairs = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4),
                    (4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]

Por ejemplo, la tupla (0, 1) indica que los científicos de datos con id 0 (Hero) e id 1 (Dunn) son amigos. La red aparece representada en la figura siguiente:   


<img src="IMAGES/redDS.jpg" width="500">

Representar las amistades como una lista de pares no es la forma más sencilla de trabajar con ellas. Para encontrar todas las amistades por usuario, hay que pasar repetidamente por cada par buscando pares que contengan 1. Si
hubiera muchos pares, el proceso tardaría mucho en realizarse.   

En lugar de ello, vamos a crear un **dict** (*dictionary*) en el que las claves sean id de usuario y los valores sean listas de id de amigos (consultar cosas en un **dict** es muy rápido).      

Aún tendremos que consultar cada par para crear el dict, pero solamente hay que hacerlo una vez y, después, las consultas no costarán nada:

In [ ]:
# Inicializar el dict con una lista vacía para cada id de usuario:
friendships = {user["id"]: [] for user in users}

# Y pasar por todos los pares de amistad para llenarlo:
for i, j in friendship_pairs:
    friendships[i].append(j)  # Añadir j como un amigo del usuario i
    friendships[j].append(i)  # Añadir i como un amigo del usuario j

Ahora que ya tenemos las amistades en un dict, podemos formular fácilmente preguntas sobre nuestro grafo, como por ejemplo: “¿Cuál es el número medio de conexiones?”.   

Primero, hallamos el número total de conexiones sumando las longitudes de todas las listas *friends*:

In [ ]:
def number_of_friends(user):
# ¿Cuántos amigos tiene este usuario?
    user_id = user["id"]
    friend_ids = friendships[user_id]
    return len(friend_ids)

total_connections = sum(number_of_friends(user)
                        for user in users)        # 24

Y, después, simplemente dividimos por el número de usuarios:

In [ ]:
num_users = len(users) # longitud de la lista de usuarios

avg_connections = total_connections / num_users # 24 / 10 == 2,4

También es sencillo encontrar las personas más conectadas (las que tienen la mayor cantidad de amigos).   

Como no hay muchos usuarios, simplemente podemos ordenarlos de “la mayor cantidad de amigos” a “la menor cantidad de amigos”:

In [ ]:
# Crea una lista (user_id, number_of_friends).
num_friends_by_id = [(user["id"], number_of_friends(user))
                     for user in users]

num_friends_by_id.sort(                                # Ordena la lista
       key=lambda id_and_friends: id_and_friends[1],   # por num_friends
       reverse=True)                                   # de mayor a menor

# Cada par es (user_id, num_friends):
# [(1, 3), (2, 3), (3, 3), (5, 3), (8, 3),
#  (0, 2), (4, 2), (6, 2), (7, 2), (9, 1)]

Una manera de pensar en lo que hemos hecho es como en una forma de identificar a las personas que son de alguna manera centrales para la red.   

En realidad, lo que acabamos de calcular es la [**métrica de la centralidad de grado**](https://es.wikipedia.org/wiki/Centralidad_de_grado) (*número de caminos de longitud 1 que lo conectan con otros nodos*) de la red:   

<img src="IMAGES/redDS2.png" width="500">

Esta métrica tiene la ventaja de ser bastante fácil de calcular, pero no siempre da los resultados que se desean o esperan. Por ejemplo, en esta red el usuario *Thor (id 4)* solo tiene dos conexiones, mientras que el usuario *Dunn (id 1)* tiene tres.   

Ahora vamos a sugerir que los usuarios podrían conocer a los amigos de sus amigos. Así que escribímos un poco de código para pasar varias veces por los amigos (iniciales) y recoger los amigos de los amigos:

In [ ]:
def foaf_ids_bad(user):
    """foaf es el acrónimo para "friend of a friend" (amigo de un amigo) """
    return [foaf_id
            for friend_id in friendships[user["id"]]
            for foaf_id in friendships[friend_id]]

Cuando aplicamos esto sobre users[0] (Hero), produce lo siguiente:
```
[0, 2, 3, 0, 1, 3]
```
- Incluye el usuario 0 dos veces, ya que *Hero* es de hecho amigo de sus dos amigos.    
- Incluye los usuarios 1 y 2, aunque ambos ya son amigos de *Hero*.    
- Y también incluye el usuario 3 dos veces, ya que se puede llegar hasta *Chi* a través de dos amigos distintos:

In [ ]:
print(friendships[0]) # [1, 2]
print(friendships[1]) # [0, 2, 3]
print(friendships[2]) # [0, 1, 3]

Saber que las personas son amigos de amigos de diversas maneras puede resultar ser información interesante, de modo que quizá en su lugar podríamos generar un contador de amigos mutuos. Y deberíamos, probablemente, excluir gente ya conocida por el usuario:

In [ ]:
from collections import Counter # no cargado inicialmente
def friends_of_friends(user):
    user_id = user["id"]
    return Counter(
        foaf_id
        for friend_id in friendships[user_id] # Para cada uno de mis amigos,
        for foaf_id in friendships[friend_id] # encuentra sus amigos
        if foaf_id != user_id                 # que no son yo 
        and foaf_id not in                    # y no son mis amigos
        friendships[user_id]
    )
print(friends_of_friends(users[3]))           # Contador({0: 2, 5: 1})

Esta función le dice correctamente a *Chi (id 3)* que tiene dos amigos mutuos con *Hero (id 0)*, pero solo uno con *Clive (id 5)*.   

### Estableciendo relaciones con colegas con intereses afines

Como científicos de datos, sabemos que también se puede disfrutar conociendo amigos con intereses comunes. Tras hacer indagaciones, conseguimos una lista de pares (user_id, interest):

In [ ]:
interests = [
(0, "Hadoop"), (0, "Big Data"), (0, "HBase"), (0, "Java"),
(0, "Spark"), (0, "Storm"), (0, "Cassandra"),
(1, "NoSQL"), (1, "MongoDB"), (1, "Cassandra"), (1, "HBase"),
(1, "Postgres"), (2, "Python"), (2, "scikit-learn"), (2, "scipy"),
(2, "numpy"), (2, "statsmodels"), (2, "pandas"), (3, "R"), (3, "Python"),
(3, "statistics"), (3, "regression"), (3, "probability"),
(4, "machine learning"), (4, "regression"), (4, "decision trees"),
(4, "libsvm"), (5, "Python"), (5, "R"), (5, "Java"), (5, "C++"),
(5, "Haskell"), (5, "programming languages"), (6, "statistics"),
(6, "probability"), (6, "mathematics"), (6, "theory"),
(7, "machine learning"), (7, "scikit-learn"), (7, "Mahout"),
(7, "neural networks"), (8, "neural networks"), (8, "deep learning"),
(8, "Big Data"), (8, "artificial intelligence"), (9, "Hadoop"),
(9, "Java"), (9, "MapReduce"), (9, "Big Data")
]

Para empezar, vemos como *Hero (id 0)* no tiene amigos comunes con *Klein (id 9)*, pero ambos comparten intereses en Java y Big Data.   

Es fácil crear una función que encuentre usuarios con un determinado interés:

In [ ]:
def data_scientists_who_like(target_interest):
    """ Localiza los ids de todos los usuarios a los que les gusta el interés objetivo. """
    return [user_id
        for user_id, user_interest in interests
        if user_interest == target_interest]

Este código funciona, pero tiene que examinar la lista completa de aficiones en cada búsqueda.    

Si tenemos muchos usuarios e intereses (o si simplemente queremos hacer muchas búsquedas), es mucho mejor que nos dediquemos a crear un índice de intereses a usuarios:

In [ ]:
from collections import defaultdict
# Las claves son intereses, los valores son listas de user_ids con ese interés
user_ids_by_interest = defaultdict(list)
for user_id, interest in interests:
    user_ids_by_interest[interest].append(user_id)

Y otro de usuarios a intereses:

In [ ]:
# Las claves son user_ids, los valores son listas de intereses para ese user_id.
interests_by_user_id = defaultdict(list)
for user_id, interest in interests:
    interests_by_user_id[user_id].append(interest)

Ahora es fácil averiguar quién tiene el mayor número de intereses en común con un determinado usuario:   

    ■ Pasamos varias veces por los intereses del usuario.
    ■ Para cada interés, volvemos a pasar en repetidas ocasiones por los demás usuarios que tienen ese mismo interés.
    ■ Contamos las veces que vemos cada uno de los usuarios.   
    
En código:

In [ ]:
def most_common_interests_with(user):
    return Counter(
        interested_user_id
        for interest in interests_by_user_id[user["id"]]
        for interested_user_id in user_ids_by_interest[interest]
        if interested_user_id != user["id"]
    )

Después, podríamos utilizar esto para crear una función “Científicos de datos que podría conocer” más completa basándonos en una combinación de amigos mutuos e intereses comunes.

### Revisando los salarios   

Nos acaban de proporcionar información sobre salarios. Los datos de sueldos son, por supuesto, confidenciales, pero los datos proporcionados son un conjunto de datos anónimo que contiene el *salario (salary)* de cada usuario (en dólares) y su *antigüedad en el puesto (tenure)* como científico de datos (en años):

In [ ]:
salaries_and_tenures = [(83000, 8.7), (88000, 8.1),
                        (48000, 0.7), (76000, 6),
                        (69000, 6.5), (76000, 7.5),
                        (60000, 2.5), (83000, 10),
                        (48000, 1.9), (63000, 4.2)]

Si trazamos una gráfica (veremos más adelante como hacerlo) obtendríamos algo parecido a esto:

<img src="IMAGES/grafica.png" width="700">

Parece claro que la gente con más experiencia tiende a ganar más.    

¿Cómo se puede convertir esto en un dato curioso?    

Lo primero que se nos ocurre es mirar el salario medio por antigüedad:

In [ ]:
# Las claves son años, los valores son listas de los salarios por antigüedad.
salary_by_tenure = defaultdict(list)

for salary, tenure in salaries_and_tenures:
    salary_by_tenure[tenure].append(salary)
    
# Las claves son años, cada valor es el salario medio para dicha antigüedad.
average_salary_by_tenure = {
    tenure: sum(salaries) / len(salaries)
    for tenure, salaries in salary_by_tenure.items()
}

Resulta que esto no es especialmente útil, ya que ninguno de los usuarios tiene la misma antigüedad en el puesto de trabajo, lo que significa que simplemente estamos informando de los salarios individuales de los usuarios:   

```
{0.7: 48000.0,
1.9: 48000.0,
2.5: 60000.0,
4.2: 63000.0,
6: 76000.0,
6.5: 69000.0,
7.5: 76000.0,
8.1: 88000.0,
8.7: 83000.0,
10: 83000.0}
```

Podría ser más útil poner los años de antigüedad en un *bucket*:

In [ ]:
def tenure_bucket(tenure):
    if tenure < 2:
     return "menos que dos"
    elif tenure < 5:
      return "entre dos y cinco"
    else:
     return "más de cinco"

Entonces podemos agrupar los salarios correspondientes a cada *bucket*:

In [ ]:
# Las claves son buckets de años de antigüedad, los valores son listas de salarios para bucket
salary_by_tenure_bucket = defaultdict(list)
for salary, tenure in salaries_and_tenures:
    bucket = tenure_bucket(tenure)
    salary_by_tenure_bucket[bucket].append(salary)

Y, por último, calcular el salario medio para cada grupo:

In [ ]:
# Las claves son buckets de años de antigüedad, los valores son el salario medio para bucket
average_salary_by_bucket = {
    tenure_bucket: sum(salaries) / len(salaries)
    for tenure_bucket, salaries in salary_by_tenure_bucket.items()
}

Lo que es más interesante:
```
{‘entre dos y cinco’: 61500.0,
‘menos que dos’: 48000.0,
‘más de cinco’: 79166.66666666667}
```

Y ya tenemos nuestra proclama: “Los científicos de datos con más de cinco años de experiencia ganan un 65 % más que los científicos de datos con poca experiencia o ninguna”.   

Pero hemos elegido los *buckets* de una forma bastante aleatoria. Lo que realmente haríamos es hacer alguna declaración sobre el efecto que tiene en el salario (en promedio) tener un año adicional de experiencia. Además de conseguir un dato curioso más eficiente, esto nos permite hacer predicciones sobre salarios que no conocemos.

Quiere entender mejor qué usuarios pagan por las cuentas y cuáles no (ella conoce sus nombres, pero esa información no es especialmente procesable).Se da cuenta de que parece haber una correspondencia entre los años de
experiencia y las cuentas de pago:   

```
0.7 paid
1.9 unpaid
2.5 paid
4.2 unpaid
6.0 unpaid
6.5 unpaid
7.5 unpaid
8.1 unpaid
8.7 paid
10.0 paid   
```
Los usuarios con muy pocos y muchos años de experiencia tienden a pagar; los usuarios con cantidades de experiencia medias no lo hacen. Según esto, si quería crear un modelo (aunque sin duda no son datos suficientes en
los que basarlo), podría intentar predecir “de pago” para usuarios con muy pocos y muchos años de experiencia y “no de pago” para usuarios con cantidades de experiencia medias:

In [ ]:
def predict_paid_or_unpaid(years_experience):
    if years_experience < 3.0:
        return "paid"
    elif years_experience < 8.5:
        return "unpaid"
    else:
        return "paid"

Por supuesto, esto lo hemos calculado a ojo.   

Con más datos (y más matemáticas), podríamos crear un modelo que predijera la probabilidad de que un usuario pagara, basándonos en sus años de experiencia.

Ya disponemos de los datos sin procesar del proyecto del sugeridor
de amigos:
```
interests = [
    (0, "Hadoop"), (0, "Big Data"), (0, "HBase"), (0, "Java"),
    (0, "Spark"), (0, "Storm"), (0, "Cassandra"),
    (1, "NoSQL"), (1, "MongoDB"), (1, "Cassandra"), (1, "HBase"),
    (1, "Postgres"), (2, "Python"), (2, "scikit-learn"), (2, "scipy"),
    (2, "numpy"), (2, "statsmodels"), (2, "pandas"), (3, "R"), (3, "Python"),
    (3, "statistics"), (3, "regression"), (3, "probability"),
    (4, "machine learning"), (4, "regression"), (4, "decision trees"),
    (4, "libsvm"), (5, "Python"), (5, "R"), (5, "Java"), (5, "C++"),
    (5, "Haskell"), (5, "programming languages"), (6, "statistics"),
    (6, "probability"), (6, "mathematics"), (6, "theory"),
    (7, "machine learning"), (7, "scikit-learn"), (7, "Mahout"),
    (7, "neural networks"), (8, "neural networks"), (8, "deep learning"),
    (8, "Big Data"), (8, "artificial intelligence"), (9, "Hadoop"),
    (9, "Java"), (9, "MapReduce"), (9, "Big Data")
]
```
Una manera sencilla (aunque no especialmente apasionante) de encontrar
los intereses más populares es contando las palabras:
1. Ponemos en minúsculas todos los hobbies (ya que habrá usuarios que los pongan en mayúscula y otros en minúscula).
2. Los dividimos en palabras.
3. Contamos los resultados.   

En código:

In [ ]:
words_and_counts = Counter(word
    for user, interest in interests
    for word in interest.lower().split())

Así es posible hacer fácilmente un listado con las palabras que aparecen más de una vez:

In [ ]:
for word, count in words_and_counts.most_common():
    if count > 1:
        print(word, count)

Lo que da los resultados esperados (a menos que se suponga que “scikit-learn” ha quedado dividido en dos palabras, en cuyo caso no los da).   
```
    learning 3
    java 3
    python 3
    big 3
    data 3
    hbase 2
    regression 2
    cassandra 2
    statistics 2
    probability 2
    hadoop 2
    networks 2
    machine 2
    neural 2
    scikit-learn 2
    r 2
```


Y hasta aquí esta breve introducción al Análisis de Datos. Más adelante veremos formas más sofisticadas de extraer temas de datos.